# LookupPipeline – Webhook JSON → CSV with Dictionary Lookup

This pipeline listens for HTTP POST requests at `/webhook`, decodes the JSON
payload, replaces the key `foo` using a simple in-memory lookup table, and
appends the transformed record to a CSV file.

---

### Imports and Setup

In [ ]:
from bspump.jupyter import *
import bspump.http.web.source
from bspump.file import FileCSVSink
from bspump.abc.lookup import Lookup
import json
import pathlib


BitSwan BSPump version devel


```websource.WebHookSource``` handles inbound webhooks;

```FileCSVSink``` writes rows;

```Lookup``` lets us register reusable lookup tables.


----------------
### Static lookup table & registration

In [2]:
words = {
    "bap": "lol",
    "baz": "baf",
}

class WordLookup(Lookup):
    def get(self, key):
        return words.get(key)

register_lookup(WordLookup)

A trivial mapping from one word to another, wrapped in a BsPump Lookup so the
pipeline can fetch it by name.

---

### Sample events (optional, for interactive notebook testing)

In [5]:
sample_events([
    b"""{"foo":"bap"}""",
    b"""{"foo":"baz"}"""
])

---
### Generator: JSON decode + lookup replace

In [ ]:
class LookupGenerator(bspump.Generator):
    """
    • Decodes the incoming bytes to a Python dictionary
    • Replaces event["foo"] via WordLookup
    • Passes the modified dictionary downstream
    """
    def __init__(self, app, pipeline, id=None, config=None):
        super().__init__(app, pipeline, id, config)
        svc = app.get_service("bspump.PumpService")
        self.word_lookup = svc.locate_lookup("WordLookup")
    
    async def generate(self, context, event, depth):
        parsed = json.loads(event.decode("utf-8"))
        parsed["foo"] = self.word_lookup.get(parsed["foo"])
        print("Emitingh:", parsed)
        await self.Pipeline.inject(context, parsed, depth)

@register_generator
def create_lookup_generator(app, pipeline):
    return LookupGenerator(app, pipeline)

{'foo': 'lol'}
{'foo': 'baf'}


#### Small utility to validate out the path where we store the .csv

In [ ]:
csv_path = pathlib.Path("examples/Lookups/data/lookup_output.csv")
csv_path.parent.mkdir(parents=True, exist_ok=True)

----
### Pipeline Definition (```auto_pipeline```)

In [ ]:

auto_pipeline(
    source=lambda app, pipeline: bspump.http.web.source.WebHookSource(
        app, pipeline,
        config={
            "port": 8080,            # integer port
            "path": "/webhook/",     # route, with trailing slash
            "secret_qparam": "not-secure"
        }
    ),
    sink=lambda app, pipeline: FileCSVSink(
        app, pipeline,
        config={"path": str(csv_path)}
    ),
    name="LookupPipeline"
)



One call wires **source** → **generator** (above) → **sink**.